In [20]:
# create tables
import dask.dataframe as dd
import pandas as pd 

d = {'col1': [1, 2, 3, 4], 'col2': [5, 6, 7, 8]}
df1 = dd.from_pandas(pd.DataFrame(data=d), npartitions=2)

In [21]:
dd.to_parquet(df=df1,
              path='hdfs://bu-23-115:9000/mania/test1.parquet')

In [43]:
lineitem = dd.read_csv('hdfs://bu-23-115:9000/tpch/in/lineitem/lineitem.tbl.*', 
                       delimiter='|', 
                       names=['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 
                              'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 
                              'l_returnflag', 'l_linestatus', 'l_shipdate', 
                              'l_commitdate', 'l_receiptdate', 'l_shipinstruct',
                              'l_shipmode', 'l_comment']) 

In [44]:
custom_sum = dd.Aggregation(
    name='custom_sum',
    chunk=lambda s: s.sum(),
    agg=lambda s0: s0.sum())

custom_mean = dd.Aggregation(
     name='custom_mean',
     chunk=lambda s: (s.count(), s.sum()),
     agg=lambda count, sum: (count.sum(), sum.sum()),
     finalize=lambda count, sum: sum / count )  # doctest: +SKIP


custom_decrease = dd.Aggregation(
    name='custom_decrease',
    chunk=lambda x, y: (x, (1 - y)),
    agg=lambda x0, y0: (x0, y0),
    finalize=lambda x0, y0: x0*y0 )

custom_increase = dd.Aggregation(
    name='custom_increase',
    chunk=lambda x, y: (x, (1 + y)),
    agg=lambda x0, y0: (x0, y0),
    finalize=lambda x0, y0: x0*y0 )


custom_sum = dd.Aggregation(
    name='custom_sum',
    chunk=lambda s: s.sum(),
    agg=lambda s0: s0.sum())


lineitem.query('l_shipdate <= "1998-09-02"').groupby(by=['l_returnflag', 'l_linestatus'])\
.agg({'l_quantity': [custom_sum],
     'l_extendedprice': [custom_sum],
     'l_quantity': [custom_mean],
     'l_extendedprice': [custom_mean],
     'l_discount': [custom_mean],
     'l_quantity': ['count']})#.sort_values(by=['l_returnflag', 'l_linestatus'])

,l_quantity,l_extendedprice,l_discount
,count,custom_mean,custom_mean
npartitions=1,,,
,int64,float64,float64
,...,...,...
